In [94]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
import csv
import glob
import os
import numpy as np
import matplotlib.dates as mdates

In [95]:
# read data
df_returns = pd.read_csv('data/preprocess/merged_three_month_interbank_rates.csv')
df_returns.head()

# add carry columns (foreign return - usd return)
df_returns['carry_AUD'] = df_returns['three_month_interbank_rate_AUD'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_CHF'] = df_returns['three_month_interbank_rate_CHF'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_EUR'] = df_returns['three_month_interbank_rate_EUR'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_GBP'] = df_returns['three_month_interbank_rate_GBP'] - df_returns['three_month_interbank_rate_USD']
df_returns['carry_JPY'] = df_returns['three_month_interbank_rate_JPY'] - df_returns['three_month_interbank_rate_USD']


# create a new dataframe with carry returns
df_carry = df_returns[['date', 'carry_AUD', 'carry_CHF', 'carry_EUR', 'carry_GBP', 'carry_JPY']].copy()
df_carry.head()

,date,carry_AUD,carry_CHF,carry_EUR,carry_GBP,carry_JPY
0,2002-04-01,0.0272,-0.004094,0.015369,0.023032,-0.0177
1,2002-05-01,0.0302,-0.006565,0.016471,0.023337,-0.0174
2,2002-06-01,0.0326,-0.006604,0.016540,0.023676,-0.0172
3,2002-07-01,0.0319,-0.007869,0.016200,0.022705,-0.0170
4,2002-08-01,0.0323,-0.010617,0.016219,0.022565,-0.0166


In [96]:
# extract the rank of each currency's carry return
# for simplicity, we sort by date, drop the date, rank, and add the date back

df_carry = df_carry.sort_values('date').reset_index(drop=True) 
carry_cols = [col for col in df_carry.columns if col.startswith('carry_')]

# rank the carry returns
df_rank = df_carry[carry_cols].rank(axis=1, method='first') 
df_rank.columns = [col.replace('carry_', 'rank_') for col in df_rank.columns]

# calculate weights based on shifted ranks w_i = rank_i - (N + 1) / 2
N = len(carry_cols)
df_weights = df_rank.sub((N + 1) / 2, axis=0)
df_weights.columns = [col.replace('rank_', 'weight_') for col in df_weights.columns]

df_weights.insert(0, 'date', df_carry['date'])

print(df_carry.head())
print(df_weights.head())

         date  carry_AUD  carry_CHF  carry_EUR  carry_GBP  carry_JPY
0  2002-04-01     0.0272  -0.004094   0.015369   0.023032    -0.0177
1  2002-05-01     0.0302  -0.006565   0.016471   0.023337    -0.0174
2  2002-06-01     0.0326  -0.006604   0.016540   0.023676    -0.0172
3  2002-07-01     0.0319  -0.007869   0.016200   0.022705    -0.0170
4  2002-08-01     0.0323  -0.010617   0.016219   0.022565    -0.0166
         date  weight_AUD  weight_CHF  weight_EUR  weight_GBP  weight_JPY
0  2002-04-01         2.0        -1.0         0.0         1.0        -2.0
1  2002-05-01         2.0        -1.0         0.0         1.0        -2.0
2  2002-06-01         2.0        -1.0         0.0         1.0        -2.0
3  2002-07-01         2.0        -1.0         0.0         1.0        -2.0
4  2002-08-01         2.0        -1.0         0.0         1.0        -2.0


In [97]:
# compute weights using formula: w_i = Z * (rank_i - (N + 1) / 2)

weights = df_weights.drop(columns='date')
weights.columns = weights.columns.str.replace('weight_', '', regex=False)


print("primo normalize \n", weights.head())

def normalize_weights(row):
    pos_sum = row[row > 0].sum()
    neg_sum = row[row < 0].sum()
    Z_pos = 1 / pos_sum if pos_sum != 0 else 0
    Z_neg = 1 / -neg_sum if neg_sum != 0 else 0
    return row.apply(lambda x: x * Z_pos if x > 0 else x * Z_neg)

df_norm_weights = weights.apply(normalize_weights, axis=1)

print("dopo normalize \n", weights.head())


df_norm_weights.columns = 'norm_weights_' + df_norm_weights.columns
df_norm_weights.insert(0, 'date', df_carry['date'])
df_norm_weights['date'] = pd.to_datetime(df_norm_weights['date'])
df_norm_weights = df_norm_weights.set_index('date')

print("finale \n", df_norm_weights.head())

df_norm_weights.to_csv("data/q6/norm_weights.csv")


primo normalize 
    AUD  CHF  EUR  GBP  JPY
0  2.0 -1.0  0.0  1.0 -2.0
1  2.0 -1.0  0.0  1.0 -2.0
2  2.0 -1.0  0.0  1.0 -2.0
3  2.0 -1.0  0.0  1.0 -2.0
4  2.0 -1.0  0.0  1.0 -2.0
dopo normalize 
    AUD  CHF  EUR  GBP  JPY
0  2.0 -1.0  0.0  1.0 -2.0
1  2.0 -1.0  0.0  1.0 -2.0
2  2.0 -1.0  0.0  1.0 -2.0
3  2.0 -1.0  0.0  1.0 -2.0
4  2.0 -1.0  0.0  1.0 -2.0
finale 
             norm_weights_AUD  norm_weights_CHF  norm_weights_EUR  \
date                                                               
2002-04-01          0.666667         -0.333333               0.0   
2002-05-01          0.666667         -0.333333               0.0   
2002-06-01          0.666667         -0.333333               0.0   
2002-07-01          0.666667         -0.333333               0.0   
2002-08-01          0.666667         -0.333333               0.0   

            norm_weights_GBP  norm_weights_JPY  
date                                            
2002-04-01          0.333333         -0.666667  
2002-05-

In [107]:
df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv", parse_dates=['date'], index_col='date')

print("hedged returns \n", df_hedged_returns.head())

print("france = ",df_hedged_returns['france_hedged_return_usd'].mean())
print("germany = ",df_hedged_returns['germany_hedged_return_usd'].mean())
# choosing france for hedging in EUR since it has a better mean hedged return than germany


df_carry_returns = df_hedged_returns.join(df_norm_weights, how='inner')

df_carry_returns['australia_carry_return'] = df_carry_returns['australia_hedged_return_usd'] * df_carry_returns['norm_weights_AUD']
df_carry_returns['switzerland_carry_return'] = df_carry_returns['switzerland_hedged_return_usd'] * df_carry_returns['norm_weights_CHF']
df_carry_returns['france_carry_return'] = df_carry_returns['france_hedged_return_usd'] * df_carry_returns['norm_weights_EUR']
df_carry_returns['uk_carry_return'] = df_carry_returns['uk_hedged_return_usd'] * df_carry_returns['norm_weights_GBP']
df_carry_returns['japan_carry_return'] = df_carry_returns['japan_hedged_return_usd'] * df_carry_returns['norm_weights_JPY']
df_carry_returns['strategy_carry_return'] = df_carry_returns[['australia_carry_return', 'switzerland_carry_return', 'france_carry_return', 'uk_carry_return', 'japan_carry_return']].sum(axis=1)

df_carry_returns = df_carry_returns.drop(columns=[col for col in df_carry_returns.columns if 'norm_weights' in col or 'hedged_return' in col])
df_carry_returns.to_csv("data/q6/carry_returns.csv")

print("carry returns \n", df_carry_returns.head())

carry_cols = [col for col in df_carry_returns.columns if col.endswith('_carry_return')]
total_returns = df_carry_returns[carry_cols].sum()

print("Total Carry Returns:\n", total_returns)


hedged returns 
             australia_hedged_return_usd  france_hedged_return_usd  \
date                                                                
2002-05-01                    -0.092040                 -0.121464   
2002-06-01                    -0.064245                 -0.175409   
2002-07-01                    -0.021001                 -0.137967   
2002-08-01                    -0.002708                 -0.007809   
2002-09-01                    -0.087387                 -0.166910   

            germany_hedged_return_usd  japan_hedged_return_usd  \
date                                                             
2002-05-01                  -0.119228                -0.011291   
2002-06-01                  -0.163781                -0.138615   
2002-07-01                  -0.142610                -0.073287   
2002-08-01                  -0.006058                 0.015716   
2002-09-01                  -0.223067                 0.031508   

            switzerland_hedged_retur

In [ ]:
# Q6 b

def return_stats(returns, periods_per_year=12, risk_free_rate=0):
    mean = returns.mean()
    std = returns.std()

    sharpe = ((mean * periods_per_year) - risk_free_rate) / (std * np.sqrt(periods_per_year))

    return mean, std, sharpe

print("Long leg = UK, Hhort legs = AUD, CHF, JPY\n")

uk_stats = return_stats(df_carry_returns['uk_carry_return'])
print(f"UK Carry Return :\n Mean: {uk_stats[0]}, Std: {uk_stats[1]}, Sharpe Ratio: {uk_stats[2]}")

australia_stats = return_stats(df_carry_returns['australia_carry_return'])
print(f"\nAustralia Carry Return :\n Mean: {australia_stats[0]}, Std: {australia_stats[1]}, Sharpe Ratio: {australia_stats[2]}")
switzerland_stats = return_stats(df_carry_returns['switzerland_carry_return'])
print(f"Switzerland Carry Return :\n Mean: {switzerland_stats[0]}, Std: {switzerland_stats[1]}, Sharpe Ratio: {switzerland_stats[2]}")
japan_stats = return_stats(df_carry_returns['japan_carry_return'])
print(f"Japan Carry Return :\n Mean: {japan_stats[0]}, Std: {japan_stats[1]}, Sharpe Ratio: {japan_stats[2]}")

print("Full strategy stats:\n")
strategy_stats = return_stats(df_carry_returns['strategy_carry_return'])
print(f"Strategy Carry Return :\n Mean: {strategy_stats[0]}, Std: {strategy_stats[1]}, Sharpe Ratio: {strategy_stats[2]}")



Long leg = UK, Hhort legs = AUD, CHF, JPY

UK Carry Return :
 Mean: 0.00063892281339373, Std: 0.018200144806856087, Sharpe Ratio: 0.12160856813577764

Australia Carry Return :
 Mean: -0.00843206517468951, Std: 0.03306792045801215, Sharpe Ratio: -0.8833192467508632
Switzerland Carry Return :
 Mean: -0.008822769309982981, Std: 0.027826851498000477, Sharpe Ratio: -1.0983265361119274
Japan Carry Return :
 Mean: -0.008637591084979737, Std: 0.03444381129342119, Sharpe Ratio: -0.8687044814373611
Full strategy stats:

Strategy Carry Return :
 Mean: -0.028639906153244216, Std: 0.047083163661018924, Sharpe Ratio: -2.1071554553371312


" # uk as the long leg and the rest as short legs\nmean_long = df_carry_returns['uk_carry_return'].mean()\nstd_long = df_carry_returns['uk_carry_return'].std()\n\nmean_short = df['short_leg'].mean()\nstd_short = df['short_leg'].std()\n\nmean_strat = df['strategy_return'].mean()\nstd_strat = df['strategy_return'].std()\n\n# Annualized Sharpe Ratio (assumes monthly data)\nrf = 0  # Assuming risk-free rate = 0\nsharpe_long = (mean_long - rf) / std_long * np.sqrt(12)\nsharpe_short = (mean_short - rf) / std_short * np.sqrt(12)\nsharpe_strat = (mean_strat - rf) / std_strat * np.sqrt(12) "